In [7]:
import pandas as pd 
import numpy as np
import os 

from password import password
from sqlalchemy import create_engine

In [8]:
Original = pd.read_csv("MergedDF_notclean.csv")

In [9]:
Original = Original.dropna()

In [10]:
IncidentCount = Original.groupby('State')[['Incident Date']].count()
IncidentCount = IncidentCount.rename(columns={"Incident Date": "# of Incidents"})

In [11]:
DFT = Original.drop(columns={"Unnamed: 0", "Incident Date", "City Or County", "Address"})

In [12]:
DFTS = DFT.groupby('State')[['# Killed','# Injured']].sum()

In [13]:
DFTS.head()

,# Killed,# Injured
State,,
Alabama,17.0,47.0
Arizona,7.0,26.0
Arkansas,3.0,20.0
Connecticut,1.0,12.0
Delaware,0.0,10.0


In [14]:
LawsandRegistration = pd.read_csv("GunLawsandRegistration.csv")

In [15]:
LawsandRegistration1 = LawsandRegistration.drop(columns="Unnamed: 0")
LawsandRegistration1.head()

,State,firearmRegRequired,permitReqToCarry,permitReqToPurchase,openCarryLegal,Number of Registered Weapons
0,Washington,True,True,False,True,"119,829"
1,New York,True,True,True,False,"82,917"
2,New Jersey,True,True,True,False,"90,217"
3,Michigan,True,True,False,True,"83,355"
4,Maryland,True,True,True,True,"128,289"


In [16]:
Merge = pd.merge(IncidentCount, DFTS, how="left", on="State")
Merge.head()

,# of Incidents,# Killed,# Injured
State,,,
Alabama,12,17.0,47.0
Arizona,6,7.0,26.0
Arkansas,5,3.0,20.0
Connecticut,3,1.0,12.0
Delaware,2,0.0,10.0


In [17]:
FinalMerge = pd.merge(Merge, LawsandRegistration1, how="left", on="State")
FinalMerge.head()

,State,# of Incidents,# Killed,# Injured,firearmRegRequired,permitReqToCarry,permitReqToPurchase,openCarryLegal,Number of Registered Weapons
0,Alabama,12,17.0,47.0,False,True,False,True,"168,265"
1,Arizona,6,7.0,26.0,False,False,False,True,"204,817"
2,Arkansas,5,3.0,20.0,False,False,False,True,"108,801"
3,Connecticut,3,1.0,12.0,True,True,True,True,"74,877"
4,Delaware,2,0.0,10.0,False,True,False,True,"5,281"


In [18]:
FinalC = FinalMerge.rename(columns={"firearmRegRequired":"Fire Arm Registration Required","permitReqToCarry":"Permit to Carry Required","permitReqToPurchase":"Permit to Purchase Required",  "openCarryLegal":"Open Carry is Legal",  })

In [19]:
FinalClean = FinalC.rename(columns={"# of Incidents": "Mass Shootings 2019", "# Killed": "Total Killed", "# Injured": "Total Injured"})

In [20]:
FinalClean.head()

,State,Mass Shootings 2019,Total Killed,Total Injured,Fire Arm Registration Required,Permit to Carry Required,Permit to Purchase Required,Open Carry is Legal,Number of Registered Weapons
0,Alabama,12,17.0,47.0,False,True,False,True,"168,265"
1,Arizona,6,7.0,26.0,False,False,False,True,"204,817"
2,Arkansas,5,3.0,20.0,False,False,False,True,"108,801"
3,Connecticut,3,1.0,12.0,True,True,True,True,"74,877"
4,Delaware,2,0.0,10.0,False,True,False,True,"5,281"


In [21]:
Final = FinalClean.set_index("State")
Final

Final.to_csv("FinalClean.csv")

In [32]:
SQLReady = Final.rename(columns={"Mass Shootings 2019":"mass_shootings_2019", "Total Killed": "total_killed", "Total Injured":"total_injured", "Fire Arm Registration Required": "fire_arm_registration_required", "Permit to Carry Required": "permit_to_carry_required", "Permit to Purchase Required":"permit_to_purchase_required", "Open Carry is Legal":"open_carry_is_legal", "Number of Registered Weapons":"number_of_registered_weapons"})

In [33]:
SQLReady.to_csv("SQLReady.csv")

In [34]:
SQLReady.head()

,mass_shootings_2019,total_killed,total_injured,fire_arm_registration_required,permit_to_carry_required,permit_to_purchase_required,open_carry_is_legal,number_of_registered_weapons
State,,,,,,,,
Alabama,12,17.0,47.0,False,True,False,True,"168,265"
Arizona,6,7.0,26.0,False,False,False,True,"204,817"
Arkansas,5,3.0,20.0,False,False,False,True,"108,801"
Connecticut,3,1.0,12.0,True,True,True,True,"74,877"
Delaware,2,0.0,10.0,False,True,False,True,"5,281"


In [35]:
connection_string = f"postgres:{password}@localhost:5432/MassShootingData"
engine = create_engine(f"postgresql://{connection_string}")

In [36]:
engine.table_names()

['massshootingdata']

In [39]:
SQLReady.to_sql(name='massshootingdata', con=engine, if_exists='replace', index=True)